In [1]:
from pyspark import SparkContext
import json

sc = SparkContext()

22/05/15 10:55:49 WARN Utils: Your hostname, alejandro-hpelitebook840g2 resolves to a loopback address: 127.0.1.1; using 10.8.22.181 instead (on interface wlo1)
22/05/15 10:55:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/15 10:55:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Viaje de ejemplo
s = '{ "_id" : { "$oid" : "5cf83b752f3843a016be4e2f" }, "user_day_code" : "e4d55deb9ac172a8d8f5f0a32599815bd51b7c8760d67e42b11adf7c0829341b", "idplug_base" : 21, "user_type" : 1, "idunplug_base" : 8, "travel_time" : 219, "idunplug_station" : 90, "ageRange" : 0, "idplug_station" : 66, "unplug_hourTime" : { "$date" : "2019-06-01T00:00:00.000+0200" }, "zip_code" : "" }'
data = json.loads(s)
data

{'_id': {'$oid': '5cf83b752f3843a016be4e2f'},
 'user_day_code': 'e4d55deb9ac172a8d8f5f0a32599815bd51b7c8760d67e42b11adf7c0829341b',
 'idplug_base': 21,
 'user_type': 1,
 'idunplug_base': 8,
 'travel_time': 219,
 'idunplug_station': 90,
 'ageRange': 0,
 'idplug_station': 66,
 'unplug_hourTime': {'$date': '2019-06-01T00:00:00.000+0200'},
 'zip_code': ''}

In [17]:
# RDD base y extraccion de datos

text_sample = './05_bicimad/sample_10e3.json'

rdd_base = sc.textFile(text_sample)

def mapper(line):
    data = json.loads(line)
    # u_t = data['user_type'] # No se necesita por ahora (?)
    u_c = data['user_day_code']
    start = data['idunplug_station']
    end = data['idplug_station']
    time = data['travel_time']
    date = data['unplug_hourTime']
    return u_c, start, end, time, date

rdd = rdd_base.map(mapper)
rdd.count()

1000

## Análisis 1

Lista de viajes en la que se sale y llega al mismo sitio.
Salen y vuelven a la misma estación.

**Problema**: Ver personas que lo usan con el proposito de dar un paseo

*Datos necesarios*:
- 'user_day_code'
- 'id(un)plug_station'

In [10]:
# Filtrar cogiendo solo los que cumplan que "start = end"
rdd_ciclos = rdd.filter(lambda x: x[1] == x[2])

In [11]:
rdd_ciclos.count()

22

In [12]:
rdd_ciclos.collect()

[('bdfbb7774e97c37e95c79efa9e5f1cba2f6672dbf1f71c1faa2bc89e9e423ae0',
  83,
  83,
  357,
  {'$date': '2019-06-01T00:00:00.000+0200'}),
 ('c199289b53d6909ee2943de60baf3fd465f9ce30251683764ed30febe54b29ef',
  48,
  48,
  520,
  {'$date': '2019-06-01T00:00:00.000+0200'}),
 ('d13cad5bd193759211b8c179566df5bce0d60530fe3901ceb6c777353a8dd0b6',
  58,
  58,
  431,
  {'$date': '2019-06-01T00:00:00.000+0200'}),
 ('bf2b917be58d51896b51e3892b7c6cf4fa9d10e56e645be45a5328f8b5648d40',
  39,
  39,
  921,
  {'$date': '2019-06-01T00:00:00.000+0200'}),
 ('72c4d5ae9ffe74250e24f097428e68286a2508dfa3c5a3edb18a5778c917680b',
  42,
  42,
  954,
  {'$date': '2019-06-01T00:00:00.000+0200'}),
 ('f9f2fc299f6d7c7f6e2305eb757a8737c4a24396b8512af9ad800d3dc192eb21',
  106,
  106,
  2256,
  {'$date': '2019-06-01T00:00:00.000+0200'}),
 ('619aa4734bf1670c0746fef949f7608234983082331854d4b6a54b9053c34234',
  106,
  106,
  2266,
  {'$date': '2019-06-01T00:00:00.000+0200'}),
 ('78544a2c490c114d10242b8d28c624cd1e9551e177869f

## Análisis 2

Tiempo medio de viaje.
  Media de la duración por viaje individual.

Uso medio de los usuarios.
  Media de todos uso por identificación.

*Datos necesarios*:
- 'user_day_code'
- 'id(un)plug_station'
- 'travel_time'

Mas ideas:
- Ranking de viajes mas largos y mas cortos en tiempo.
- Mediana y otras medidas estadisticas (?)

### Tiempo medio por viaje

In [18]:
rdd_travelTime = rdd.map(lambda x: x[3])

total_travels = rdd_travelTime.count()
total_travelTime = rdd_travelTime.sum()
average_travelTime = total_travelTime/total_travels

print(f'viajes: {total_travels}')
print(f'tiempo: {total_travelTime}')
print(f'media por viaje: {average_travelTime} segundos (?)')

viajes: 1000
tiempo: 904235
media por viaje: 904.235 segundos (?)


### Tiempo medio por identificador (persona)

In [25]:
rdd_travelTime_id = rdd.map(lambda x: (x[0], x[3]))\
                        .groupByKey()\
                        .mapValues(sum)\
                        .map(lambda x: x[1])

total_travels_id = rdd_travelTime_id.count()
total_travelTime_id = rdd_travelTime_id.sum()
average_travelTime_id = total_travelTime_id/total_travels_id

print(f'viajes por id: {total_travels_id}')
print(f'tiempo: {total_travelTime_id}')
print(f'media por viaje: {average_travelTime_id} segundos/persona (?)')

viajes por id: 926
tiempo: 904235
media por viaje: 976.4956803455724 segundos/persona (?)


In [ ]:
"""
Analisis ?:
Gráfica con usos por cada mes. Tanto por viajes como por persona (es decir, con identificador único)

  Datos necesarios:
    - 'user_day_code'
    - 'unplug_hourTime'
"""